In [ ]:
import pandas as pd
import json

with open('text.json', 'r') as file:
    data = json.load(file)

In [3]:
# 기본 형식 점수 계산

# 보조 용언 구성 제거
def is_real_VP(morp_result):
    return (
        'VX' not in morp_result or
        'VV' in morp_result or
        'VA' in morp_result or
        'VCP' in morp_result or
        'VCN' in morp_result
    )

basic_per_sent= [] # 각 문장에 포함된 각 절이 가지는 문장 성분
for sentence in data['return_object']['sentence']:
    basic_per_clause = [] # 각 절이 가지는 문장 성분
    dep_id2label = {int(dep['id']): dep['label'] for dep in sentence['dependency']} # id값을 label값으로 치환
    for morp, dep in zip(sentence['morp_eval'], sentence['dependency']):
        if ('VP' in dep['label']) or ('VNP' in dep['label']): # 동사구/형용사구나 지정사구에 해당하는 어절 찾기
            if is_real_VP(morp['result']): # 보조 용언 구성 제거
                basic_elements = [dep_id2label[int(sub)] for sub in dep['mod'] if int(sub) in dep_id2label]
                basic_per_clause.append(basic_elements) # 각 절의 문장 성분을 저장
    basic_per_sent.append(basic_per_clause) # 각 문장에 포함된 각 절의 문장 성분을 저장

basic_score_per_sent = [] # 각 문장의 기본 형식 점수
for sent in basic_per_sent:
    basic_score = 0 # 특정 문장의 기본 형식 점수
    for clause in sent:
        basic_score_per_clause = 1 # 절당 기본 형식 점수(최소 1점)
        if ('VP_OBJ' in clause) or ('NP_OBJ' in clause) or ('X_OBJ' in clause) or ('VNP_OBJ' in clause): # 목적어 찾기
            basic_score_per_clause += 1
        if ('AP_CMP' in clause) or ('X_CMP' in clause) or ('NP_CMP' in clause) or ('VP_CMP' in clause) or ('VNP_CMP' in clause): # 보어 찾기
            basic_score_per_clause += 1
        basic_score += basic_score_per_clause # 절당 기본 형식 점수 합산
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if basic_score > 20:
        basic_score = 20
    else:
        basic_score = basic_score
    basic_score_per_sent.append(basic_score) # 기본 형식 점수 저장

basic_score_per_sent

[7]

In [4]:
# 수식 구조 점수 계산

add1_score_per_sent = [] # 각 문장의 수식 구조 점수

for sentence in data['return_object']['sentence']:
    add1_score = 0 # 특정 문장의 수식 구조 점수
    for dep in sentence['dependency']:
        if dep['label'] in ['NP_MOD', 'X_MOD', 'DP', 'NP_AJT', 'AP_AJT', 'X_AJT', 'NP_INT', 'IP']: # 관형어/부사어/독립어 찾기
            add1_score += 1
        if dep['label'] == 'AP': # 없이/달리/같이 부사절 제외
            if dep['text'] not in ['없이', '달리', '같이']:
                add1_score += 1
            elif len(dep['mod']) == 0:
                add1_score += 1
        if dep['label'] in ['NP_SBJ', 'NP_OBJ', 'NP_CMP', 'NP_MOD', 'NP_CNJ', 'NP', 'NP_AJT', 'NP_INT']: # 명사 관형어 찾기
            add1_score += [dep_id2label[x] for x in dep['mod']].count('NP')
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if add1_score >= 14:
        add1_score = 14
    else:
        add1_score = add1_score
    add1_score_per_sent.append(add1_score/2) # 수식 구조 점수 저장

add1_score_per_sent

[1.0]

In [5]:
# 내포 구조 점수 계산

clause_list = ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP'] # 내포절의 유형

add2_score_per_sent = [] # 각 문장의 내포 구조 점수
for sentence in data['return_object']['sentence']:
    dep_id2label = {int(dep['id']): dep['label'] for dep in sentence['dependency']} # id값을 label값으로 치환
    add2_score = 0 # 특정 문장의 내포 구조 점수
    for morp, dep in zip(sentence['morp_eval'], sentence['dependency']):
        # 명사절
        if (dep['label'] in ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
                # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                pre_pre_add2_elements = [dep_id2label[int(n)] for n in x['mod']]
                                if pre_pre_add2_elements.count('VP_SBJ') + pre_pre_add2_elements.count('VNP_SBJ') + pre_pre_add2_elements.count('X_SBJ') + pre_pre_add2_elements.count('NP_SBJ') >= 2:
                                    add2_score += 1

        # 인용절
        if (dep['label'] in ['VP_CMP', 'VNP_CMP']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
            # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                pre_pre_add2_elements = [dep_id2label[int(n)] for n in x['mod']]
                                if pre_pre_add2_elements.count('VP_SBJ') + pre_pre_add2_elements.count('VNP_SBJ') + pre_pre_add2_elements.count('X_SBJ') + pre_pre_add2_elements.count('NP_SBJ') >= 2:
                                    add2_score += 1

        # 관형절
        if (dep['label'] in ['VP_MOD', 'VNP_MOD']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
                # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                pre_pre_add2_elements = [dep_id2label[int(n)] for n in x['mod']]
                                if pre_pre_add2_elements.count('VP_SBJ') + pre_pre_add2_elements.count('VNP_SBJ') + pre_pre_add2_elements.count('X_SBJ') + pre_pre_add2_elements.count('NP_SBJ') >= 2:
                                    add2_score += 1

        # 부사절
        if (dep['label'] in ['VP_AJT', 'VNP_AJT']):
            # 한 단어
            if len(dep['mod']) == 0:
                add2_score += 1
            else:
                add2_score += 2
                # 내포절 안의 내포절
                pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                pre_pre_add2_elements = [dep_id2label[int(n)] for n in x['mod']]
                                if pre_pre_add2_elements.count('VP_SBJ') + pre_pre_add2_elements.count('VNP_SBJ') + pre_pre_add2_elements.count('X_SBJ') + pre_pre_add2_elements.count('NP_SBJ') >= 2:
                                    add2_score += 1

            # 부사절(달리, 없이, 같이)
        if (dep['label'] in ['AP']) & (dep['text'] in ['달리', '없이', '같이']) & (len(dep['mod']) >= 1):
            add2_score += 2
            # 내포절 안의 내포절
            pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
            for clause in clause_list:
                add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
            for num, elem in zip(dep['mod'], pre_add2_elements):
                if elem in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN', 'VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                pre_pre_add2_elements = [dep_id2label[int(n)] for n in x['mod']]
                                if pre_pre_add2_elements.count('VP_SBJ') + pre_pre_add2_elements.count('VNP_SBJ') + pre_pre_add2_elements.count('X_SBJ') + pre_pre_add2_elements.count('NP_SBJ') >= 2:
                                    add2_score += 1

            # 서술절
        if (dep['label'] in ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']):
            pre_add2_elements = [p['label'] for sub in dep['mod'] for p in sentence['dependency'] if int(sub) == int(p['id'])]
            if pre_add2_elements.count('VP_SBJ') + pre_add2_elements.count('VNP_SBJ') + pre_add2_elements.count('X_SBJ') + pre_add2_elements.count('NP_SBJ') >= 2:
                add2_score += 2
            # 내포절 안의 내포절
                for clause in clause_list:
                    add2_score += pre_add2_elements.count(clause)
                # 내포절 안의 서술절
                for num, elem in zip(dep['mod'], pre_add2_elements):
                    if elem in ['VP_SBJ', 'VNP_SBJ', 'VP_OBJ', 'VNP_OBJ', 'VP_CNJ', 'VNP_CNJ', 'VP_MOD', 'VNP_MOD',
               'VP_AJT', 'VNP_AJT', 'VP_CMP', 'VNP_CMP']:
                        for x in sentence['dependency']:
                            if int(num) == int(x['id']):
                                pre_pre_add2_elements = [dep_id2label[int(n)] for n in x['mod']]
                                if pre_pre_add2_elements.count('VP_SBJ') + pre_pre_add2_elements.count('VNP_SBJ') + pre_pre_add2_elements.count('X_SBJ') + pre_pre_add2_elements.count('NP_SBJ') >= 2:
                                    add2_score += 1
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if add2_score > 17:
        add2_score = 17
    else:
        add2_score = add2_score
    add2_score_per_sent.append(add2_score)

add2_score_per_sent


[5]

In [7]:
# 접속 구조 점수 계산

add3_score_per_sent = [] # 각 문장의 접속 구조 점수

for sentence in data['return_object']['sentence']:
    add3_score = 0 # 특정 문장의 접속 구조 점수
    for morp, dep, num in zip(sentence['morp_eval'], sentence['dependency'], range(len(sentence['dependency']))):
        if (dep['label'] in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN']) and (dep['head'] != -1): # 접속절 찾기
            if ('VX' not in morp['result']) or ('VV' in morp['result']) or ('VA' in morp['result']) or ('VCP' in morp['result']) or ('VCN' in morp['result']): # 보조 용언 구성 제외
                if num < len(sentence['morp_eval']) - 1:
                    if ('VX' in sentence['morp_eval'][num+1]['result']) and ('VV' not in sentence['morp_eval'][num+1]['result']) and ('VA' not in sentence['morp_eval'][num+1]['result']) and ('VCP' not in sentence['morp_eval'][num+1]['result']) and ('VCN' not in sentence['morp_eval'][num+1]['result']):
                        if (sentence['dependency'][num+1]['label'] in ['VP', 'VNP', 'VP_PRN', 'VNP_PRN']) and (sentence['dependency'][num+1]['head'] != -1):
                            add3_score += 1
                    else:
                        add3_score += 1
    # 최대 점수 제한 기능(삭제를 원할 경우 아래 4줄 삭제)
    if add3_score > 7:
        add3_score = 7
    else:
        add3_score = add3_score
    add3_score_per_sent.append(add3_score) # 접속 구조 점수 저장

add3_score_per_sent


[1]

In [8]:
# 합산 점수 계산
sum_score_per_sent = [a + b + c + d for a, b, c, d in zip(basic_score_per_sent, add1_score_per_sent, add2_score_per_sent, add3_score_per_sent)]
sum_score_per_sent

[14.0]

In [9]:
sentence_list = [x['text'] for x in data['return_object']['sentence']]
df_result_final = pd.DataFrame({'문장': sentence_list,
                                '기본 형식': basic_score_per_sent,
                                '수식': add1_score_per_sent,
                                '내포': add2_score_per_sent,
                                '접속': add3_score_per_sent,
                                '합산': sum_score_per_sent})

df_result_final

,문장,기본 형식,수식,내포,접속,합산
0,상처를 보고 텔레비전 프로그램에 제보해야겠다고 생각했다”고 증언했다.,7,1.0,5,1,14.0


In [ ]:
df_result_final.to_excel('df_result.xlsx')